<a href="https://colab.research.google.com/github/LucjanSakowicz/data-science-projects/blob/main/titanic/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Load data

In [4]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

Print basic train data info

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Print first few rows.

In [6]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Name column seems widely unorganized, we'll try to fix that.

First will extract part before coma, which looks like surname.

In [7]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [8]:
df_train[['Surname', 'FirstNames']] = df_train['Name'].str.split(',', expand=True)
df_train[['Surname', 'FirstNames']]

,Surname,FirstNames
0,Braund,Mr. Owen Harris
1,Cumings,Mrs. John Bradley (Florence Briggs Thayer)
2,Heikkinen,Miss. Laina
3,Futrelle,Mrs. Jacques Heath (Lily May Peel)
4,Allen,Mr. William Henry
...,...,...
886,Montvila,Rev. Juozas
887,Graham,Miss. Margaret Edith
888,Johnston,"Miss. Catherine Helen ""Carrie"""
889,Behr,Mr. Karl Howell


Split First names into title, Names and others.

In [9]:
df_train[['Title', 'Names', 'Other']] = df_train['FirstNames'].str.split('.', expand=True)
df_train.drop(columns=['FirstNames', 'Name'], inplace=True)

In [10]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names,Other
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Braund,Mr,Owen Harris,None
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Cumings,Mrs,John Bradley (Florence Briggs Thayer),None
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Heikkinen,Miss,Laina,None
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Futrelle,Mrs,Jacques Heath (Lily May Peel),None
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Allen,Mr,William Henry,None


What exactly is in Other column?

In [11]:
df_train['Other'].value_counts()

 Barrett)    1
Name: Other, dtype: int64

In [12]:
df_train[df_train['Other'].isna() == False]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names,Other
513,514,1,1,female,54.0,1,0,PC 17603,59.4,NaN,C,Rothschild,Mrs,Martin (Elizabeth L,Barrett)


Remove non-required column

In [13]:
df_train.iloc[513, -2] = df_train.iloc[513, -2] + df_train.iloc[513,-1]
df_train.drop(columns='Other', inplace=True)
print(df_train)

     PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch  \
0              1         0       3    male  22.0      1      0   
1              2         1       1  female  38.0      1      0   
2              3         1       3  female  26.0      0      0   
3              4         1       1  female  35.0      1      0   
4              5         0       3    male  35.0      0      0   
..           ...       ...     ...     ...   ...    ...    ...   
886          887         0       2    male  27.0      0      0   
887          888         1       1  female  19.0      0      0   
888          889         0       3  female   NaN      1      2   
889          890         1       1    male  26.0      0      0   
890          891         0       3    male  32.0      0      0   

               Ticket     Fare Cabin Embarked    Surname  Title  \
0           A/5 21171   7.2500   NaN        S     Braund     Mr   
1            PC 17599  71.2833   C85        C    Cumings    Mrs   
2    S

Trim strings from blank characters

In [14]:
df_train[['Surname', 'Title', 'Names']] = df_train[['Surname', 'Title', 'Names']].apply(lambda x: x.str.strip())

In [15]:
df_train['Surname'].value_counts()

Andersson    9
Sage         7
Panula       6
Skoog        6
Carter       6
            ..
Hanna        1
Lewy         1
Mineff       1
Haas         1
Dooley       1
Name: Surname, Length: 667, dtype: int64

Let's make bar chart for better visualisation of surnames

In [16]:
px.bar(x=df_train['Surname'].value_counts().index, y=df_train['Surname'].value_counts(), template='plotly_dark', title='Surname occurrences',
       width=950, color_discrete_sequence=['#03fcb5'])

Some of the Surnames looks like have more than one word and some have characters which are not letters, let's take a look

In [17]:
temp_surname = df_train['Surname'].str.split(' ', expand=True).apply(lambda x: x.str.strip())
print(temp_surname)

             0     1     2
0       Braund  None  None
1      Cumings  None  None
2    Heikkinen  None  None
3     Futrelle  None  None
4        Allen  None  None
..         ...   ...   ...
886   Montvila  None  None
887     Graham  None  None
888   Johnston  None  None
889       Behr  None  None
890     Dooley  None  None

[891 rows x 3 columns]


In [18]:
temp_surname[temp_surname[2].isna() == False]

,0,1,2
170,Van,der,hoef
307,Penasco,y,Castellana
505,Penasco,y,Castellana
547,Padro,y,Manent
866,Duran,y,More


"Vad der hoef" can be loosely translated "from hoef", so it should be treated as surname in our case

The two surnames may be connected by the letter "y" which means "and." For example, Juan Gomez y Conde has Juan as a first name, Gomez as the surname of the father, and Conde as the surname of the mother. Let's check if any of these exist in "Surname" column

In [19]:
y_names = temp_surname[(temp_surname[2].isna() == False) & (temp_surname[1] == 'y')]
df_train[df_train['Surname'] == y_names[2].str]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names


Let's check another column for values not equal None

In [20]:
temp_surname[temp_surname[1].isna() == False]

,0,1,2
18,Vander,Planke,None
38,Vander,Planke,None
153,van,Billiard,None
170,Van,der,hoef
200,Vande,Walle,None
282,de,Pelsmaeker,None
286,de,Mulder,None
307,Penasco,y,Castellana
333,Vander,Planke,None
355,Vanden,Steen,None


Again, let's check to be sure that there isn't person with Surname of a second column

In [21]:
y_names = temp_surname[(temp_surname[1].isna() == False)]
df_train[df_train['Surname'] == y_names[1].str]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names


Let's check non letter values

In [22]:
df_train[~df_train['Surname'].str.isalpha()]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names
18,19,0,3,female,31.0,1,0,345763,18.0000,NaN,S,Vander Planke,Mrs,Julius (Emelia Maria Vandemoortele)
28,29,1,3,female,NaN,0,0,330959,7.8792,NaN,Q,O'Dwyer,Miss,"Ellen ""Nellie"""
38,39,0,3,female,18.0,2,0,345764,18.0000,NaN,S,Vander Planke,Miss,Augusta Maria
39,40,1,3,female,14.0,1,0,2651,11.2417,NaN,C,Nicola-Yarred,Miss,Jamila
47,48,1,3,female,NaN,0,0,14311,7.7500,NaN,Q,O'Driscoll,Miss,Bridget
49,50,0,3,female,18.0,1,0,349237,17.8000,NaN,S,Arnold-Franchi,Mrs,Josef (Josefine Franchi)
125,126,1,3,male,12.0,1,0,2651,11.2417,NaN,C,Nicola-Yarred,Master,Elias
153,154,0,3,male,40.5,0,2,A/5. 851,14.5000,NaN,S,van Billiard,Mr,Austin Blyler
170,171,0,1,male,61.0,0,0,111240,33.5000,B19,S,Van der hoef,Mr,Wyckoff
184,185,1,3,female,4.0,0,2,315153,22.0250,NaN,S,Kink-Heilmann,Miss,Luise Gretchen


Let's check dashed surnames which might be surname given at marriage

In [23]:
y_names = df_train[df_train['Surname'].str.contains("-")]['Surname'].str.split('-', expand=True).apply(lambda x: x.str.strip())
y_names

,0,1
39,Nicola,Yarred
49,Arnold,Franchi
125,Nicola,Yarred
184,Kink,Heilmann
192,Andersen,Jensen
351,Williams,Lambert
353,Arnold,Franchi
430,Bjornstrom,Steffansson
444,Johannesen,Bratthammer
587,Frolicher,Stehli


Check if any other person has such surname

In [24]:
df_train[df_train['Surname'] == y_names[0].str]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names


In [25]:
df_train[df_train['Surname'] == y_names[1].str]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names


It seems that nothing else can be done for Surname, let's move to Title column

In [38]:
df_train['Title'].value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: Title, dtype: int64

In [39]:
px.bar(x=df_train['Title'].value_counts().index, y=df_train['Title'].value_counts(), template='plotly_dark', title='Title occurrences',
       width=950, color_discrete_sequence=['#03fcb5'])

Ovewhelming part of titles belong to 'Mr', 'Miss', 'Mrs' and 'Master' class. Let's check how big is this part

In [40]:
len(df_train[(df_train['Title'] == 'Mr') | (df_train['Title'] == 'Miss') | (df_train['Title'] == 'Mrs') | (df_train['Title'] == 'Master')]) / len(df_train)

0.9696969696969697

Let's check what is percentage of survival based on most common titles

In [41]:
print(f"Mr:   {len(df_train[(df_train['Title'] == 'Mr') & df_train['Survived'] == 1.0]) / len(df_train[(df_train['Title'] == 'Mr') & ~df_train['Survived'].isna()]) }")
print(f"Miss:   {len(df_train[(df_train['Title'] == 'Miss') & df_train['Survived'] == 1.0]) / len(df_train[(df_train['Title'] == 'Miss') & ~df_train['Survived'].isna()]) }")
print(f"Mrs:   {len(df_train[(df_train['Title'] == 'Mrs') & df_train['Survived'] == 1.0]) / len(df_train[(df_train['Title'] == 'Mrs') & ~df_train['Survived'].isna()]) }")
print(f"Master:   {len(df_train[(df_train['Title'] == 'Master') & df_train['Survived'] == 1.0]) / len(df_train[(df_train['Title'] == 'Master') & ~df_train['Survived'].isna()]) }")

Mr:   0.15667311411992263
Miss:   0.6978021978021978
Mrs:   0.792
Master:   0.575


Check if there might be Surnames hidden in the Names column which might be related

In [43]:
df_train['Names']

0                                Owen Harris
1      John Bradley (Florence Briggs Thayer)
2                                      Laina
3              Jacques Heath (Lily May Peel)
4                              William Henry
                       ...                  
886                                   Juozas
887                           Margaret Edith
888                 Catherine Helen "Carrie"
889                              Karl Howell
890                                  Patrick
Name: Names, Length: 891, dtype: object

In [68]:
names = df_train['Names']\
.apply(lambda x: x.replace("(", ""))\
.apply(lambda x: x.replace(")", ""))\
.apply(lambda x: x.replace('"', ""))\
.str.split(' ', expand=True)\
.apply(lambda x: x.str.strip())
print(names)

             0        1         2       3       4     5     6     7     8  \
0         Owen   Harris      None    None    None  None  None  None  None   
1         John  Bradley  Florence  Briggs  Thayer  None  None  None  None   
2        Laina     None      None    None    None  None  None  None  None   
3      Jacques    Heath      Lily     May    Peel  None  None  None  None   
4      William    Henry      None    None    None  None  None  None  None   
..         ...      ...       ...     ...     ...   ...   ...   ...   ...   
886     Juozas     None      None    None    None  None  None  None  None   
887   Margaret    Edith      None    None    None  None  None  None  None   
888  Catherine    Helen    Carrie    None    None  None  None  None  None   
889       Karl   Howell      None    None    None  None  None  None  None   
890    Patrick     None      None    None    None  None  None  None  None   

        9  
0    None  
1    None  
2    None  
3    None  
4    None  
.. 

In [94]:
exist = names.isin(df_train['Surname']).any(1)
print(exist[exist == True])

427    True
dtype: bool


<ipython-input-94-62a390d50a49>:1: FutureWarning:

In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.



In [99]:
df_train.iloc[427, :]

PassengerId                                                  428
Survived                                                       1
Pclass                                                         2
Sex                                                       female
Age                                                         19.0
SibSp                                                          0
Parch                                                          0
Ticket                                                    250655
Fare                                                        26.0
Cabin                                                        NaN
Embarked                                                       S
Surname                                                 Phillips
Title                                                       Miss
Names          Kate Florence ("Mrs Kate Louise Phillips Marsh...
Name: 427, dtype: object

In [100]:
names.iloc[427, :]

0        Kate
1    Florence
2         Mrs
3        Kate
4      Louise
5    Phillips
6    Marshall
7        None
8        None
9        None
Name: 427, dtype: object

It seems like nothing worthy hides in Names. Let's crosscheck Surnames with SibSp and Cabin to check if all related passengers were in the same cabin

In [256]:
multi_occurr_surnames_without_sibsp = df_train[(df_train['Surname'].isin(df_train['Surname'].value_counts().loc[lambda x: x > 1].index.to_list()))]\
         .groupby('Surname')\
         .sum()\
         .loc[lambda x: x['SibSp'] > 0]\
         .sort_values(by='Surname').index.to_list()

<ipython-input-256-13575f064f2d>:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [257]:
non_compliant_cabin_surnames = df_train[(df_train['Surname'].isin(multi_occurr_surnames_without_sibsp))]\
         .groupby('Surname')['Cabin'].nunique()
non_compliant_cabin_surnames = non_compliant_cabin_surnames.index[non_compliant_cabin_surnames.gt(1)].tolist()
non_compliant_cabin_surnames

['Andrews', 'Brown', 'Duff Gordon', 'Hays', 'Newell', 'Taussig', 'Thayer']

In [259]:
df_train[df_train['Surname'].isin(non_compliant_cabin_surnames)].sort_values(by='Surname')

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title,Names
275,276,1,1,female,63.0,1,0,13502,77.9583,D7,S,Andrews,Miss,Kornelia Theodosia
806,807,0,1,male,39.0,0,0,112050,0.0000,A36,S,Andrews,Mr,Thomas Jr
194,195,1,1,female,44.0,0,0,PC 17610,27.7208,B4,C,Brown,Mrs,James Joseph (Margaret Tobin)
684,685,0,2,male,60.0,1,1,29750,39.0000,NaN,S,Brown,Mr,Thomas William Solomon
670,671,1,2,female,40.0,1,1,29750,39.0000,NaN,S,Brown,Mrs,Thomas William Solomon (Elizabeth Catherine Ford)
345,346,1,2,female,24.0,0,0,248733,13.0000,F33,S,Brown,Miss,"Amelia ""Mildred"""
599,600,1,1,male,49.0,1,0,PC 17485,56.9292,A20,C,Duff Gordon,Sir,"Cosmo Edmund (""Mr Morgan"")"
556,557,1,1,female,48.0,1,0,11755,39.6000,A16,C,Duff Gordon,Lady,"(Lucille Christiana Sutherland) (""Mrs Morgan"")"
820,821,1,1,female,52.0,1,1,12749,93.5000,B69,S,Hays,Mrs,Charles Melville (Clara Jennings Gregg)
310,311,1,1,female,24.0,0,0,11767,83.1583,C54,C,Hays,Miss,Margaret Bechstein


Again, it seems like nothing interesting here - it seems that there are no people who are related but have no cabin added.